In [55]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd

speech_links = pd.read_csv(r'C:\Users\FYD\Documents\GitHub\lda-brazilian-books\src\books\discursos-presidenciais\discursos-presidenciais-links.txt',
                             delimiter='|')

speech_links.head(2)

,president,link
0,fhc,/pt-BR/discursos-artigos-e-entrevistas-categor...
1,lula,/pt-BR/discursos-artigos-e-entrevistas-categor...


In [57]:
start = time.time()

BASE_URL = 'http://www.itamaraty.gov.br'

full_link = []
speech = []
for link in speech_links['link']:
    full_link.append(BASE_URL + link)
    
    response = requests.get(BASE_URL + link)
    soup = BeautifulSoup(response.text, 'html.parser')
    speech_text = soup.find("div", itemprop="articleBody")
    
    speech.append(speech_text)
    
print ("Process time: " + str((time.time() - start)))

Process time: 407.8306894302368


In [164]:
df_speech_consolidated = pd.concat([speech_links,
                                   pd.DataFrame(full_link),
                                   pd.DataFrame(speech)], axis=1)

df_speech_consolidated.columns = ['president', 'link', 'full_link', 'text']
df_speech_consolidated.head(5)

,president,link,full_link,text
0,fhc,/pt-BR/discursos-artigos-e-entrevistas-categor...,http://www.itamaraty.gov.br/pt-BR/discursos-ar...,"<div itemprop=""articleBody""> <div class=""plain..."
1,lula,/pt-BR/discursos-artigos-e-entrevistas-categor...,http://www.itamaraty.gov.br/pt-BR/discursos-ar...,"<div itemprop=""articleBody""> <p style=""text-al..."
2,lula,/pt-BR/discursos-artigos-e-entrevistas-categor...,http://www.itamaraty.gov.br/pt-BR/discursos-ar...,"<div itemprop=""articleBody""> <p style=""text-al..."
3,lula,/pt-BR/discursos-artigos-e-entrevistas-categor...,http://www.itamaraty.gov.br/pt-BR/discursos-ar...,"<div itemprop=""articleBody""> <p style=""text-al..."
4,lula,/pt-BR/discursos-artigos-e-entrevistas-categor...,http://www.itamaraty.gov.br/pt-BR/discursos-ar...,"<div itemprop=""articleBody""> <p style=""text-al..."


In [165]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]"\|@,;]')

from nltk.corpus import stopwords

#BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('portuguese'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = str(text)
    text = text.lower() # lowercase text
    text = text.replace("<div itemprop=\"articleBody\">", "")
    text = text.replace("<div class=\"plain\" id=\"parent-fieldname-text\">", "")
    text = text.replace("<p style=\"text-align: justify;\">", "")
    text = text.replace("<br/>", "")
    text = text.replace("</div>", "")
    text = text.replace("\n", "")
    text = text.replace(" < p>", "")    
    text = text.replace(".< p>", " ")
    text = text.replace(".", " ")
    text = text.replace("<div itemprop=\"articlebody\">", "")
    text = text.replace("<", "")
    text = text.replace(">", "")
     
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    #text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
    
df_speech_consolidated['text_processed'] = df_speech_consolidated['text'].apply(clean_text)

In [166]:
df_speech_consolidated.to_csv(r'C:\Users\FYD\Documents\GitHub\lda-brazilian-books\src\books\discursos-presidenciais\df_speech_consolidated.csv',
                             sep='|',
                             index_label=None)

In [167]:
df_speech_consolidated['text_processed'].head(1)[0]

'desejo antes tudo agradecer presidente jorge sampaio esposa maria josé ritta gesto receber palácio ajuda demonstração fidalguia distingue estimado casal amigo brasil amigo brasileiros convivência presidente sampaio faz nutrir apreço cada vez maior sítios históricos portugal sinto cativo palácio ajuda onde evoca nascimento rei comum d joão vi percebo próximo palácio bélem hospedou passagem ano mostra sobre carta pêro vaz caminha cujo catálogo satisfação prefaciar juntamente presidente sampaio ressaltei prefácio prestígio hoje cerca carta dom manuel i decorre apenas dotes narrativos escrivão resulta sobretudo fato encontro etnias descrito caminha vingou vingou bem dando margem universo civilizatório profícuos caracterizado ambos lados atlântico plasticidade cultural propensão assimilar influências diversos povos sobre atualidade lastro cultural une brasil portugal gostaria dizer algumas palavras hoje noite inspirado anfitriões jorge sampaio maria ritta tanto cultivam legado humanista ja

In [168]:
# Basic counters
print(f'Qty rows: {df_speech_consolidated.shape[0]}, Qty columns: {df_speech_consolidated.shape[1]}')

Qty rows: 398, Qty columns: 5


In [169]:
# Show stats about the language per artist
df_speech_consolidated.groupby(['president']).size().reset_index()

,president,0
0,bolsonaro,5
1,dilma,191
2,fhc,1
3,lula,122
4,temer,79


In [177]:
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pyLDAvis
import pyLDAvis.sklearn
import random
import seaborn as sns
import spacy
import string
from collections import Counter
from PIL import Image
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from spacy.lang.en import English
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

# Generate graphs inline in Jupyter
%matplotlib inline
    
# Lock random seeds used by libraries
random.seed(42)
np.random.seed(42)
    
# Define function to cleanup text by removing 
# personal pronouns, stopwords, and puncuation
nlp = spacy.load("pt_core_news_sm")
punctuations = string.punctuation

ModuleNotFoundError: No module named 'spacy'

In [ ]:
!pip install spacy

In [ ]:
!python -m spacy download pt_core_news_sm

In [ ]:
# Data exploration in some specific class to see the most frequent words
def get_word_frequency(artist):

    # Word Frequency per Category
    def cleanup_text(docs, logging=False):
        texts = []
        counter = 1
        for doc in docs:
            if counter % 1000 == 0 and logging:
                print("Processed %d out of %d documents." % (counter, len(docs)))
            counter += 1
            doc = nlp(doc, disable=['parser', 'ner'])
            tokens = [tok.lemma_.lower().strip() for tok in doc if tok.lemma_ != '-PRON-']
            tokens = [tok for tok in tokens if tok not in stoplist and tok not in punctuations]
            tokens = ' '.join(tokens)
            texts.append(tokens)
        return pd.Series(texts)

    df_text = [text for text in df_raw_lyrics[df_raw_lyrics['artist'] == artist]['lyric']]
    df_text_clean = cleanup_text(df_text)
    df_text_clean = ' '.join(df_text_clean).split()
    df_text_clean_counts = Counter(df_text_clean)
    df_common_words = [word[0] for word in df_text_clean_counts.most_common(31)]
    df_common_counts = [word[1] for word in df_text_clean_counts.most_common(31)]
    df_common_words.pop(0)
    df_common_counts.pop(0)

    fig = plt.figure(figsize=(18,6))
    sns.barplot(x=df_common_words, y=df_common_counts)
    plt.title(f'Most Common Words used by {artist}')
    plt.xticks(rotation=45)
    plt.show()
    
    fig.savefig(f'word_frequency_{artist}.png', format='png', dpi=500)